In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sante\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sante\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tfIDFdict = {}
file = open("tfIDF.txt", encoding="utf8")
strings = file.read().splitlines()
for string in strings:
    tfIDFdict[string.split()[0]] = string.split()[2]
allWords = [*tfIDFdict]

In [3]:
dictionary = {}
for i in range(129):
    number = "%(i)03d "%{"i": i}
    number = number[:-1]
    for word in allWords:
        if dictionary.get(number) is None:
            dictionary[number] = [0]
        else:
            dictionary[number].append(0)

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
r = re.compile("[а-яА-Я]+")
for i in range(129):
    number = "%(i)03d "%{"i": i}
    number = number[:-1]
    f = open('/Users/sante/Выкачка'+number+'.txt', encoding="utf-8")
    for line in f:
        tokens = tokenizer.tokenize(line)
        filteredTokens = []
        for token in tokens:
            if token not in russian_stopwords:
                filteredTokens.append(token)
        russian = [morph.parse(w)[0].normal_form for w in filter(r.match, filteredTokens)]
        for word in russian:
            if tfIDFdict.get(word) is not None:
                dictionary[number][allWords.index(word)] = tfIDFdict[word]

In [11]:
import math
lengthDict = {}
for i in range(129):
    number = "%(i)03d "%{"i": i}
    number = number[:-1]
    res = 0
    for value in dictionary[number]:
        res += float(value) * float(value)
    lengthDict[number] = math.sqrt(res)

In [12]:
linksDict = {}
file = open("index.txt", encoding="utf8")
strings = file.read().splitlines()
for string in strings:
    linksDict[string.split()[0]] = string.split()[1]
    
def searchFor(query):
    queryValues = []
    for word in allWords:
        queryValues.append(0)
    queryLength = 0
    for value in query.split():
        if tfIDFdict.get(morph.parse(value)[0].normal_form) is not None:
            normalWord = morph.parse(value)[0].normal_form
            queryValues[allWords.index(normalWord)] = tfIDFdict[normalWord]
    for value in queryValues:
        queryLength += float(value) * float(value)
    queryLength = math.sqrt(queryLength)
    finalValues = {}
    for key, values in dictionary.items():
        chisl = 0
        for pair in enumerate(values):
            chisl += float(pair[1]) * float(queryValues[pair[0]])
        znam = lengthDict[key] * queryLength
        if znam == 0:
            znam = 1
        finalValues[key] = chisl / znam
    soretdDict = dict(reversed(sorted(finalValues.items(), key=lambda item: item[1])))
    for index in [*soretdDict][:10]:
        print(linksDict[index])

In [14]:
searchFor("Евклид математик")

https://ru.wikipedia.org/wiki/Евклид
https://ru.wikipedia.org/wiki/Гаусс,_Карл_Фридрих
https://ru.wikipedia.org/wiki/Дюрер,_Альбрехт
https://ru.wikipedia.org/wiki/Архимед
https://ru.wikipedia.org/wiki/Эйлер,_Леонард
https://ru.wikipedia.org/wiki/Ньютон,_Исаак
https://ru.wikipedia.org/wiki/Эйнштейн,_Альберт
https://ru.wikipedia.org/wiki/Википедия:Избранные_статьи
https://ru.wikipedia.org/wiki/Брахмагупта
https://ru.wikipedia.org/wiki/Бернерс-Ли,_Тим
